In [2]:
import numpy as np 
import pandas as pd 
import math

In [4]:
#Importing the libraries for the garphs
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go

#importing the libraries for datapeprocessing
from sklearn.preprocessing import LabelEncoder,StandardScaler

#importing the libraries for splitting the data
from sklearn.model_selection import train_test_split

#Libraries for model building
from sklearn import linear_model
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

#import the libraries for checking the metrics
from sklearn.metrics import r2_score,mean_squared_error

reading the input files

In [5]:
df_train = pd.read_csv("airbnb_listing_train.csv")
df_validate = pd.read_csv("airbnb_listing_validate.csv")
df_ss=pd.read_csv("sample_submission.csv")

Explanatory Data Analysis ...Knowing about the data

In [6]:
def aboutdf (df):
    count_null = df.isnull().sum()
    df_stats = pd.DataFrame(index = df.columns, data =
                           {'datatype': df.dtypes,
                            'unique_values': df.nunique(),
                            'have_null?': df.isnull().any(),
                            'Number of null values' : count_null,
                            'percentage of null values' : count_null/df.shape[0]*100 })
    return df_stats


In [7]:
aboutdf(df_train)

,datatype,unique_values,have_null?,Number of null values,percentage of null values
id,int64,12901,False,0,0.000000
name,object,12653,True,22,0.170529
host_id,int64,11601,False,0,0.000000
host_name,object,4536,True,8,0.062011
neighbourhood_group,float64,0,True,12901,100.000000
neighbourhood,object,22,False,0,0.000000
latitude,float64,5227,False,0,0.000000
longitude,float64,7887,False,0,0.000000
room_type,object,4,False,0,0.000000
minimum_nights,int64,56,False,0,0.000000


Dropping the neighbourhood_group column as it has 100 percent null values 

In [8]:
df_train=df_train.drop(columns='neighbourhood_group',axis=1)

In [9]:
bool_series = pd.notnull(df_train["name"])
df_train[bool_series].name


0                      Luxury home 100m2+garden in center!
1                              East is where your home is!
2          Amsterdam Central - LUXURY house in BUSY STREET
3                               great appartment down town
4            Private room with canal view in peaceful area
                               ...                        
12896    Quiet stylish private room 15 mins to city  ce...
12897    Beautiful, spacious apt. with great views @ IJ...
12898                  Great apartment de Pijp/Albert Cuyp
12899             Stylish apartment in an amazing location
12900                     Amsterdam Apartments First Floor
Name: name, Length: 12879, dtype: object

In [10]:
df_train.name.describe()

count         12879
unique        12653
top       Amsterdam
freq             28
Name: name, dtype: object

There are 12653 unique airbnb apartments/rooms

In [11]:
df_train[bool_series].groupby(['name']).size().sort_values(ascending=False).reset_index(name='count').head(100)

,name,count
0,Amsterdam,28
1,Spacious apartment near Vondelpark,6
2,Lovely apartment in Amsterdam,5
3,Spacious apartment in Amsterdam,5
4,Cosy apartment in the city centre of Amsterdam,4
...,...,...
95,Home sweet home,2
96,Spacious private room in the heart of Amsterdam,2
97,Light appartment in Amsterdam,2
98,NEW! Temporary housing in the centre of Amsterdam,2


In [12]:
df_train.host_name.describe()

count       12893
unique       4536
top       Martijn
freq           76
Name: host_name, dtype: object

In [13]:
bool_series1 = pd.notnull(df_train["host_name"])
df_train[bool_series1].host_name


0        Marjolein
1              Ana
2             Mark
3            Ayden
4           Alissa
           ...    
12896     Michelle
12897         Rens
12898     Francien
12899         Olaf
12900         City
Name: host_name, Length: 12893, dtype: object

In [14]:
df_train[bool_series1].groupby(['host_name']).size().sort_values(ascending=False).reset_index(name='count').head(100)


,host_name,count
0,Martijn,76
1,Anne,58
2,Thomas,53
3,Tim,51
4,Marieke,51
...,...,...
95,Sara,20
96,Steven,20
97,Floor,20
98,Dennis,20


As you can see in the graph above Martijn has hosted 76 times ...

Now lets check about neighbourhood

In [15]:
df_train.neighbourhood.describe()

count                      12901
unique                        22
top       De Baarsjes - Oud-West
freq                        2209
Name: neighbourhood, dtype: object

In [16]:
df_train.groupby(['neighbourhood']).size().sort_values(ascending=False).reset_index(name='count')

,neighbourhood,count
0,De Baarsjes - Oud-West,2209
1,De Pijp - Rivierenbuurt,1600
2,Centrum-West,1429
3,Centrum-Oost,1097
4,Westerpark,958
5,Zuid,924
6,Oud-Oost,831
7,Bos en Lommer,730
8,Oostelijk Havengebied - Indische Buurt,644
9,Watergraafsmeer,381


De Baarsjes - Oud-West is the most selected neigbourhood by the airbnd customers.

lets know check about the room_type details

In [17]:
df_train[['name','host_name','price']].sort_values(by='price',ascending=False)

,name,host_name,price
12839,"Center, Large&Sunny Room,Pvt Bathrm",David,9000
6023,Moie kamer in Amsterdam,Mery,7550
10029,"Hotel Estheréa, Classic Room",Hotel Estheréa,7000
12885,"Hotel Estheréa, Deluxe Room",Hotel Estheréa,7000
1657,"Hotel Estheréa, Triple Room",Hotel Estheréa,7000
...,...,...,...
5427,1 Bed in 10 Bedded Girls Dorm En-suite,ClinkNOORD,20
7810,Minimalist White Room,Daniela,19
480,Appartement in het veelzijdige Amsterdam-Noord,Han,12
1887,Beautiful calm room in Amsterdam,Herry,10


In [18]:
df_train['last_review']=pd.to_datetime(df_train['last_review'])

C:\Users\balan\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15-02-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\balan\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '19-06-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\balan\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16-03-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\balan\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17-02-2020' in DD/MM/YYYY format. Provide format or specif

In [19]:
df_train['name'].fillna(df_train['name'].mode()[0], inplace=True)
df_train['host_name'].fillna(df_train['host_name'].mode()[0],inplace=True)
df_train['last_review'].fillna(df_train['last_review'].mean(),inplace=True)
df_train['reviews_per_month'].fillna(df_train['reviews_per_month'].mean(),inplace=True)

Dealing with the last_review datatime columns.split it into year,month,day

In [20]:
df_train["day"] = df_train['last_review'].map(lambda x: x.day)
df_train["month"] = df_train['last_review'].map(lambda x: x.month)
df_train["year"] = df_train['last_review'].map(lambda x: x.year)


In [21]:
aboutdf(df_train)

,datatype,unique_values,have_null?,Number of null values,percentage of null values
id,int64,12901,False,0,0.0
name,object,12653,False,0,0.0
host_id,int64,11601,False,0,0.0
host_name,object,4536,False,0,0.0
neighbourhood,object,22,False,0,0.0
latitude,float64,5227,False,0,0.0
longitude,float64,7887,False,0,0.0
room_type,object,4,False,0,0.0
minimum_nights,int64,56,False,0,0.0
number_of_reviews,int64,362,False,0,0.0


Now that we have replaces the missing values next step is to encode the object data types ....

encoding for the object columns name,host_name,neighbourhood

In [22]:
le = LabelEncoder()

In [23]:
df_train['name']=LabelEncoder().fit_transform(df_train['name'])
df_train['host_name']=LabelEncoder().fit_transform(df_train['host_name'])
df_train['neighbourhood']=LabelEncoder().fit_transform(df_train['neighbourhood'])
df_train['room_type']=LabelEncoder().fit_transform(df_train['room_type'])

check this code later

In [24]:
#X=df_train.drop(columns=['price','last_review','id','host_id'],axis=1)
X=df_train.drop(columns=['price','last_review'],axis=1)# just check feature selection
y=df_train['price']

In [25]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=2)

In [28]:
from sklearn.linear_model import LinearRegression
# Create a Logistic Regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

LinearRegression()

In [30]:
# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 60463.04668682
R-squared Score: 0.035572614956272175


Feature Selection

In [31]:
# Compare y_pred with y_test
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(comparison.head())

      Actual   Predicted
3445     150  186.692029
3553      80  108.671649
9049     136  154.897220
2057     125  252.486834
9012     189  141.400649


In [32]:
df_train.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price,day,month,year
0,11602914,7657,3123809,2856,8,52.35368,4.90177,0,3,13,2020-02-15,0.27,1,0,220,15,2,2020
1,13289321,5031,10259430,180,17,52.35728,4.92052,0,4,14,2019-06-19,0.29,1,0,110,19,6,2019
2,40779315,752,224969266,2863,5,52.37582,4.89850,0,2,9,2020-03-16,1.65,1,7,100,16,3,2020
3,7820311,12437,693472,403,20,52.37792,4.87300,0,3,42,2020-02-17,0.72,1,0,130,17,2,2020
4,27346603,9258,41888346,128,20,52.38051,4.87493,2,2,89,2020-02-26,4.02,1,24,90,26,2,2020


In [35]:
data = df_train.copy()

In [37]:
data.head()

,name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price,day,month,year
0,7657,8,52.35368,4.90177,0,3,13,2020-02-15,0.27,1,0,220,15,2,2020
1,5031,17,52.35728,4.92052,0,4,14,2019-06-19,0.29,1,0,110,19,6,2019
2,752,5,52.37582,4.89850,0,2,9,2020-03-16,1.65,1,7,100,16,3,2020
3,12437,20,52.37792,4.87300,0,3,42,2020-02-17,0.72,1,0,130,17,2,2020
4,9258,20,52.38051,4.87493,2,2,89,2020-02-26,4.02,1,24,90,26,2,2020


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Handle categorical variables using one-hot encoding
data = pd.get_dummies(data)

# Split the data into features (X) and target variable (y)
X = data.drop(['price','last_review'], axis=1)
y = data['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Calculate the root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)


RMSE: 218.60682289050698


In [40]:
# Compare y_pred with y_test
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(comparison.head())

      Actual   Predicted
6237      50   85.835982
8866     150  138.760804
4659     119  156.398952
8504     180  157.855244
4872     117  133.806782


In [41]:
# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 47788.94301428148
R-squared Score: 0.027561401015085196
